In [3]:
import json
from groq import Groq
from datetime import datetime

GROQ_MODEL = "llama-3.3-70b-versatile"

# Initialize GROQ API client
client = Groq(api_key="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")

# Session string to hold entire conversation
session_history = "User-Bot Conversation History:\n"

crop_plan = {}

current_date = datetime.today()
date = current_date.strftime("%A, %B %d, %Y")


def chat_with_llm(user_input):
    """
    Process the user's input using LLM to determine the next action and provide responses.
    """
    global session_history
    session_history += f"User: {user_input}\n"

    prompt = (
        "Use clear, concise, and easy-to-understand language keep the response as short as possible and proper format. "
        "Guide the user by asking for necessary agricultural inputs and providing a detailed crop plan. "
        "The chatbot should cover all stages from soil preparation to harvesting. "
        "Ask for the following crucial parameters: "
        "'CropType': The type of crop the user wants to cultivate. "
        "'SoilType': The type of soil available. If unsure, guide the user on how to identify it. "
        "'LandArea': Total land area available for cultivation in acres or hectares. "
        "'Location': User's location to determine the climate and suitable crops. "
        "'WaterSource': Availability and type of irrigation. "
        "'Fertilizers': Any preferences or restrictions on fertilizers. "
        "'Pesticides': Any preferences or restrictions on pesticides. "
        "'Budget': Total budget for the entire farming process. "
        "If enough information is provided, generate a complete crop plan detailing soil preparation, sowing, irrigation schedules, fertilization, pest management, and harvesting. "
        "Once the user confirms they are satisfied with the plan, respond with 'OK PLAN' without any other sentence or explanation. Ensure that the response is in this exact format only.\n\n"
        f"Conversation:\n{session_history}"
    )

    response = client.chat.completions.create(
        model=GROQ_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=False
    )

    bot_response = response.choices[0].message.content.strip()
    session_history += f"Bot: {bot_response}\n"
    return bot_response


def parse_crop_details(bot_response):
    """
    Parse the crop details from the bot response into a dictionary.
    """
    global crop_plan
    try:
        bot_response = bot_response.strip()

        if not bot_response.startswith("{") or not bot_response.endswith("}"):
            raise ValueError("Bot response is not valid JSON.")

        crop_plan = json.loads(bot_response)

    except json.JSONDecodeError as e:
        print(f"Error parsing crop details: {e}")
        crop_plan = {}
    except ValueError as e:
        print(f"Error: {e}")
        crop_plan = {}


def chatbot():
    """
    Main chatbot loop to interact with the user.
    """
    global session_history
    print("Welcome to the Agriculture Chatbot! Type 'exit' to end the session.")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye! Happy farming!")
            break

        bot_response = chat_with_llm(user_input)

        if "OK PLAN" in bot_response.upper():
            prompt_for_details = (
                "Provide crop plan details strictly in a dictionary format as follows. Do not include any additional text or explanation, only the dictionary. The key names must be the same as provided in the following format and should be in double quotes:\n"
                "'CropType': Name of the crop, "
                "'SoilPreparation': Steps and methods for soil preparation, "
                "'Sowing': Recommended sowing techniques and timeline, "
                "'Irrigation': Irrigation schedule and methods, "
                "'Fertilization': Fertilizer types and application schedule, "
                "'PestManagement': Pest control methods and schedules, "
                "'Harvesting': Harvesting techniques and timeline.\n"
                f"Conversation:\n{session_history}"
            )

            response = client.chat.completions.create(
                model=GROQ_MODEL,
                messages=[{"role": "user", "content": prompt_for_details}],
                temperature=1,
                max_tokens=1024,
                top_p=1,
                stream=False
            )

            parse_crop_details(response.choices[0].message.content.strip())
            break
        else:
            print(f"Bot: {bot_response}")


def main():
    chatbot()
    print("\nFinal Crop Plan:")
    print(json.dumps(crop_plan, indent=4))


if __name__ == "__main__":
    main()

Welcome to the Agriculture Chatbot! Type 'exit' to end the session.


You:  hii


Bot: To create a detailed crop plan, I need some information. 
1. What is your 'CropType'? 
2. What is your 'SoilType'? If unsure, you can identify it by checking its texture (clay, silt, sand) or sending a soil sample for testing.
3. What is your 'LandArea' in acres or hectares?
4. What is your 'Location' (city/state/country)?
5. What is your 'WaterSource' (well, river, rain, irrigation system)?
6. Do you have any preferences or restrictions on 'Fertilizers'?
7. Do you have any preferences or restrictions on 'Pesticides'?
8. What is your 'Budget' for the entire farming process?


You:  banana


Bot: To create a detailed crop plan for banana, I need more information. 
1. What is your 'SoilType'? If unsure, you can identify it by checking its texture (clay, silt, sand) or sending a soil sample for testing.
2. What is your 'LandArea' in acres or hectares?
3. What is your 'Location' (city/state/country)?
4. What is your 'WaterSource' (well, river, rain, irrigation system)?
5. Do you have any preferences or restrictions on 'Fertilizers'?
6. Do you have any preferences or restrictions on 'Pesticides'?
7. What is your 'Budget' for the entire farming process?


...

Bot: To create a detailed crop plan for banana, I will provide you with the following plan:

**Soil Preparation:**
Loamy soil is suitable for banana cultivation. Add 10 tons of organic compost per acre to improve soil fertility. Till the soil to a depth of 12 inches to loosen and aerate it.

**Sowing:**
Plant banana suckers 12 inches apart in rows 10 feet apart. Use 400 suckers per acre.

**Irrigation:**
Use the pump system to irrigate the field every 7-10 days during dry weather. Ensure the soil remains moist but not waterlogged.

**Fertilization:**
Apply 100 kg of NPK fertilizer (10:10:10) per acre at planting, followed by 200 kg per acre 3 months later.

**Pest Management:**
Regularly inspect the field for pests like aphids, whiteflies, and nematodes. Use neem oil or insecticidal soap to control infestations.

**Harvesting:**
Bananas will be ready to harvest in 12-14 months. Cut the stem 1 inch above the fruit, leaving the fruit intact.

**Budget Breakdown:**
- Land preparation: 10%

You:  what time of year is it best to cultivate banana?


Bot: To determine the best time for cultivating banana, I need to consider your 'Location' (Amravati, Maharashtra). In your region, the ideal time for planting banana is during the summer or rainy season, typically from June to September. This allows the plants to take advantage of the warm weather and ample moisture, promoting healthy growth and development. 

Please confirm if you want to proceed with the previously discussed plan, considering the optimal planting time. 
1. Are you satisfied with the plan?
2. Do you need any modifications?


You:  yes



Final Crop Plan:
{
    "CropType": "Banana",
    "SoilPreparation": "Add 10 tons of organic compost per acre to improve soil fertility. Till the soil to a depth of 12 inches to loosen and aerate it.",
    "Sowing": "Plant banana suckers 12 inches apart in rows 10 feet apart. Use 400 suckers per acre. Ideal planting time is from June to September.",
    "Irrigation": "Use the pump system to irrigate the field every 7-10 days during dry weather. Ensure the soil remains moist but not waterlogged.",
    "Fertilization": "Apply 100 kg of NPK fertilizer (10:10:10) per acre at planting, followed by 200 kg per acre 3 months later.",
    "PestManagement": "Regularly inspect the field for pests like aphids, whiteflies, and nematodes. Use neem oil or insecticidal soap to control infestations.",
    "Harvesting": "Bananas will be ready to harvest in 12-14 months. Cut the stem 1 inch above the fruit, leaving the fruit intact."
}
